# 07.1- Création d'un dataset de la fusion des datasets de clean metrics et clean jobs/job_events

Ce notebook génère 1 csv :

- dataset_for_preprocess.csv qui fusionne les données du dataset de metrics et jobs/job_events

Etapes : 

B) Import des datasets : 

- metrics est quasi-brut (la colonne events est fractionnée et tous les identifiants d'évènements sont des nombres)

- jobs (le dataset fusionné des données de jobs et job_events) dont chaque ligne est un job, un job est unique et il a un début et une fin

C) Réduction du dataset jobs et réductions succesives du dataset metrics (les lignes évènements avec certains identifiants)

D) Concaténation des datasets jobs et metrics (suppression des lignes inutiles : toutes celles de jobs et metrics si aucun jobId associé)


# A. Imports

## a) Librairies

In [1]:
import os, json, ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## b) Datasets

In [2]:
# source path to datasets
path = '../data/'
metrics = 'metrics/clean_merge_metrics_dataset.csv'
jobs = 'jobs/merge_raw_jobs_and_clean_jobevents_dataset.csv'

save_csv = '../data/dataset_for_preprocess_criticality_balanced_07.1.csv'

In [3]:
CRITICALITY_NULL = 'NO_EVENT'

# B. Jeux de données

### a) Metrics

In [4]:
# création d'un dataframe à partir du csv de données
metrics_df = pd.read_csv(os.path.join(path, metrics), index_col=0)
print(f'metrics dataset shape {metrics_df.shape}')

metrics dataset shape (2693836, 14)


In [5]:
# la colonne timestamp contient-elle des valeurs en double ?
metrics_df['timestamp'].duplicated().any()

True

In [6]:
metrics_df['timestamp'].min()

'2022-11-02 08:28:54.218000+00:00'

In [7]:
# Grouper les lignes par la colonne "timestamp" et obtenir les index correspondants
groupes = metrics_df.groupby('timestamp').groups
print(len(groupes.keys()))

1105867


In [8]:
metrics_df.head(3)

,source_events,timestamp,criticality_events,identification_events,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,status,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,iFoil,2022-11-02 08:28:57.675000+00:00,ERROR,358,iFoil L,iFoil,Gen. 2,Total Pages Counter,39968,User,Operator,ERR,7553,18000
1,iFoil,2022-11-02 08:28:57.675000+00:00,ERROR,358,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,39740,User,Operator,ERR,7553,18000
2,PLC,2022-11-02 08:28:57.613000+00:00,ERROR,358,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1734834,User,Operator,ERR,7553,18000


### b) Jobs

In [9]:
# création d'un dataframe à partir du csv de données
jobs_df = pd.read_csv(os.path.join(path, jobs), index_col=0)
print(f'jobs dataset shape {jobs_df.shape}')

jobs dataset shape (47073, 51)


In [10]:
# la colonne started_at contient-elle des valeurs en double ?
jobs_df['started_at'].duplicated().any()

False

In [11]:
jobs_df['started_at'].min()

'2022-02-22 09:43:04.487000+00:00'

In [12]:
jobs_df.head(3)

,total_copies_job,started_at,ended_at,paperHeight_job,paperWidth_job,scanner_mode,bars_job,varnishConsumptionVarnish_3d_job,run,total_run,...,y_cropmark2_cropmarksMode_remoteScannerRegistration,exposureTime_manualLighting_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,enable_specialSubstrate_remoteScannerRegistration,mode_remoteScannerRegistration,jobState,total_copies_event,varnishConsumptionVarnish_3d_event
0,40,2022-02-22 09:43:04.487000+00:00,2022-02-22 09:46:07.946000+00:00,470,330,3,2,4.414782,0,0,...,0,100,1500,20,20,False,3,SUCCESS,40,4.414782
1,18,2022-02-22 09:47:20.673000+00:00,2022-02-22 09:48:57.474000+00:00,470,330,3,2,3.004043,0,0,...,0,100,1500,20,20,False,3,SUCCESS,18,3.004043
2,15,2022-02-22 09:50:14.555000+00:00,2022-02-22 09:51:50.551000+00:00,470,330,3,2,2.503964,0,0,...,0,100,1500,20,20,False,3,SUCCESS,15,2.503964


Les tailles des datasets sont déséquilibrés :

- 3510431 lignes pour metrics

- 16295 lignes pour jobs

Les dates de début sont différentes :

- '2022-04-15 05:55:06.678000+00:00' pour metrics

- '2021-06-18 09:22:46.866000+00:00' pour jobs

Le dataset metrics compte 1242037 doublons pour la colonne timestamp

# C. Equilibrage des jeux de données

## a) Réduction de jobs

In [13]:
# concordance des données de temps dans un même cadre
jobs_reduced = jobs_df[jobs_df.started_at > metrics_df.timestamp.min()]
jobs_reduced.shape

(32118, 51)

## b) Réduction de metrics

In [14]:
# on remplace les valeurs de criticité nulles par 'UNDEFINED'
# ce sont le messages de metrics qui n'ont pas d'évènements
metrics_df['criticality_events'] = metrics_df['criticality_events'].fillna(CRITICALITY_NULL)

In [15]:
# on vérifie la distribution par criticité
metrics_df['criticality_events'].value_counts()

NO_EVENT    2367049
INFO         303552
ERROR         16860
WARNING        6375
Name: criticality_events, dtype: int64

In [16]:
# on détermine le nombre de lignes ERROR
errors = metrics_df['criticality_events'][metrics_df['criticality_events'] == 'ERROR'].count()
print(f"Le nombre de lignes en criticité 'ERROR' : {errors}")

Le nombre de lignes en criticité 'ERROR' : 16860


In [17]:
# def balance_dataframe_by_criticality(df, criticality, n):
#     # on filtre les lignes du df qui ont la valeur criticality dans la colonne 'criticality_events'
#     df_criticality = df[df['criticality_events'] == criticality]
#     # on selectionne un nombre aléatoire de lignes
#     df_criticality_sample = df_criticality.sample(n=n)
#     # on stoke les index des lignes du df qui ne sont pas dans le sample
#     index_to_delete = df_criticality[~df_criticality.index.isin(df_criticality_sample.index)].index
#     # suppression des lignes exclu du sample
#     return df.drop(index_to_delete)

def balance_dataframe_by_criticality(df, criticality, n):
    # Filtre les lignes du df avec la valeur de criticality dans la colonne 'criticality_events'
    df_criticality = df[df['criticality_events'] == criticality]

    # Obtient le nombre de lignes dans df_criticality
    taille_population = df_criticality.shape[0]

    # Assurez-vous que n est inférieur ou égal à la taille de la population
    n = min(n, taille_population)

    # Sélectionne un nombre aléatoire de lignes
    df_criticality_sample = df_criticality.sample(n=n)

    # Stocke les index des lignes du df qui ne sont pas dans le sample
    index_to_delete = df_criticality[~df_criticality.index.isin(df_criticality_sample.index)].index

    # Suppression des lignes exclues du sample
    return df.drop(index_to_delete)


### par criticité UNDEFINED

In [18]:
metrics_df = balance_dataframe_by_criticality(metrics_df, CRITICALITY_NULL, errors)

### par criticité INFO

In [19]:
metrics_df = balance_dataframe_by_criticality(metrics_df, 'INFO', errors)

### par criticité WARNING

In [20]:
metrics_df = balance_dataframe_by_criticality(metrics_df, 'WARNING', errors)

In [21]:
# on vérifie la distribution par criticité
metrics_df['criticality_events'].value_counts()

ERROR       16860
INFO        16860
NO_EVENT    16860
WARNING      6375
Name: criticality_events, dtype: int64

# D. Création du datatset pour le pre-processing

In [22]:
metrics_reduced = metrics_df
metrics_reduced['jobId'] = 0
metrics_reduced.shape

(56955, 15)

## a) Association d'un jobId à une ligne metrics

In [23]:
# DataFrame 1 
# avec les intervalles de temps
dataframe1 = jobs_reduced[['started_at', 'ended_at', 'jobId']]
# conversion des colonnes au format datetime
cols_to_convert = ['started_at', 'ended_at']
dataframe1[cols_to_convert] = dataframe1[cols_to_convert].apply(pd.to_datetime)
dataframe1 = dataframe1.sort_values('started_at')
dataframe1 = dataframe1.reset_index()
dataframe1.head(3)

C:\Users\Allan\AppData\Local\Temp\ipykernel_10828\1041892598.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe1[cols_to_convert] = dataframe1[cols_to_convert].apply(pd.to_datetime)


,index,started_at,ended_at,jobId
0,14955,2022-11-02 08:34:21.469000+00:00,2022-11-02 08:40:14.413000+00:00,1667378061
1,14956,2022-11-02 08:40:31.774000+00:00,2022-11-02 08:49:04.967000+00:00,1667378431
2,14957,2022-11-02 08:50:59.720000+00:00,2022-11-02 08:52:53.581000+00:00,1667379059


In [24]:
# DataFrame 2 
# avec les valeurs de date
dataframe2 = pd.DataFrame({
    'timestamp': metrics_reduced['timestamp'].values,
    'criticality_events': metrics_reduced['criticality_events'].values,
    'identification_events': metrics_reduced['identification_events'].values,
    'jobId': 0
}, index=None)
# conversion des colonnes au format datetime
dataframe2['timestamp'] = pd.to_datetime(dataframe2['timestamp'])
dataframe2 = dataframe2.sort_values('timestamp')
dataframe2 = dataframe2.reset_index()
dataframe2.head(3)

,index,timestamp,criticality_events,identification_events,jobId
0,2,2022-11-02 08:28:57.613000+00:00,ERROR,358,0
1,0,2022-11-02 08:28:57.675000+00:00,ERROR,358,0
2,1,2022-11-02 08:28:57.675000+00:00,ERROR,358,0


In [25]:
import time
# début du temps d'execution de la cellule
start = time.time()

# Parcours chaque ligne de job
for index1, row1 in dataframe1.iterrows():
    # creation des variables
    job_starts = row1.started_at.tz_localize(None)
    job_ends = row1.ended_at.tz_localize(None)

    # mois de référence
    month = job_starts.month
    if month != job_ends.month :
        dataframe2_by_month = dataframe2[(dataframe2['timestamp'].dt.month == month) | (dataframe2['timestamp'].dt.month == job_ends.month) & (dataframe2['jobId'] == 0)]
        dataframe2_by_month = dataframe2_by_month.sort_values('timestamp')
    else :
        dataframe2_by_month = dataframe2[(dataframe2['timestamp'].dt.month == month) & (dataframe2['jobId'] == 0)]
        dataframe2_by_month = dataframe2_by_month.sort_values('timestamp')

    for index2, row2 in dataframe2_by_month.iterrows():
        # creation des variables
        metrics_timestamp = row2['timestamp'].tz_localize(None)
        # si le timestamp de la lignes de row2 de metrics est compris dans l'intervalle de temps du job de row1
        if metrics_timestamp >= job_starts :
            if metrics_timestamp <= job_ends :
                dataframe2.loc[index2, 'jobId'] = dataframe1.loc[index1, 'jobId']
    # # avant de passer à la row1 suivante
    # # on affiche l'index1 toutes les 1000 lignes et son temps d'execution
    print(f'[ job index {index1} = time {time.time() - start}]') if index1 % 1000 == 0 and index1 != 0 else None


[ job index 1000 = time 283.4151420593262]
[ job index 2000 = time 525.0069441795349]
[ job index 3000 = time 746.5483283996582]
[ job index 4000 = time 946.5782556533813]
[ job index 5000 = time 1109.7836894989014]
[ job index 6000 = time 1289.2817380428314]
[ job index 7000 = time 1457.198656797409]
[ job index 8000 = time 1597.860450744629]
[ job index 9000 = time 1731.934752702713]
[ job index 10000 = time 1878.5009207725525]
[ job index 11000 = time 1998.8242650032043]
[ job index 12000 = time 2140.09636759758]
[ job index 13000 = time 2324.477144241333]
[ job index 14000 = time 2472.2887704372406]
[ job index 15000 = time 2648.9095225334167]
[ job index 16000 = time 2824.2686533927917]
[ job index 17000 = time 2965.6205048561096]
[ job index 18000 = time 3108.7507617473602]
[ job index 19000 = time 3230.6051230430603]
[ job index 20000 = time 3346.863140821457]
[ job index 21000 = time 3449.967326402664]
[ job index 22000 = time 3552.036420583725]
[ job index 23000 = time 3693.78

In [26]:
# on vérifie la distribution par criticality_events pour les lignes du dataframe2 (metrics_df) qui ont été associées à un jobId
dataframe2[dataframe2['jobId'] != 0]['criticality_events'].value_counts()

INFO        10004
ERROR        7792
NO_EVENT     7118
WARNING       831
Name: criticality_events, dtype: int64

In [27]:
print(f'Nombre de ligne de metrics : {dataframe2.shape[0]} (dont {dataframe2[dataframe2.jobId != 0].shape[0]} avec un jobId associé)')

Nombre de ligne de metrics : 56955 (dont 25745 avec un jobId associé)


## b) Fusion des données de job avec les lignes de metrics (dataframe2)

In [28]:
merged_df = jobs_reduced.merge(dataframe2, on='jobId')
merged_df.shape

(25745, 55)

La fusion on jobId permet de ne conserver que les lignes de metrics qui ont un jobId.

## c) Equilibrage des classes

In [29]:
# on détermine le nombre de lignes ERROR
merge_df_errors = merged_df['criticality_events'][merged_df['criticality_events'] == 'ERROR'].count()
print(f"Le nombre de lignes en criticité 'ERROR' : {merge_df_errors}")

Le nombre de lignes en criticité 'ERROR' : 7792


In [30]:
# réduction des autres classes par rapport à ERROR
merged_df = balance_dataframe_by_criticality(merged_df, CRITICALITY_NULL, merge_df_errors)
merged_df = balance_dataframe_by_criticality(merged_df, 'WARNING', merge_df_errors)
merged_df = balance_dataframe_by_criticality(merged_df, 'INFO', merge_df_errors)

In [31]:
# on vérifie la distribution par criticité pour les lignes du dataframe2 (metrics_df) qui ont été associées à un jobId
merged_df['criticality_events'].value_counts()

INFO        7792
ERROR       7792
NO_EVENT    7118
WARNING      831
Name: criticality_events, dtype: int64

## d) Output

In [32]:
merged_df.to_csv(path_or_buf=save_csv)